# Optimization Algorithms

We use an optimization method to update parameters and minimize the cost function J

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math
import sklearn
import sklearn.datasets

from opt_utils import load_params_and_grads, initialize_parameters, forward_propagation, backward_propagation
from opt_utils import compute_cost, predict, predict_dec, plot_decision_boundary, load_dataset
from testCases import *

plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Regular Gradient Descent

Simple gradient descent updates parameters using the derivative of the layer multiplied by a learning_rate 
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{1}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{2}$$

In [2]:
def update_parameters_with_gd(parameters, grads, learning_rate):
    """
    Upadate params with one step of Gradient Descent
    
    Arguments:
    
    parameters -- Python dicitionary containing the weight matrices and bias vectors that need to be updated
    
    grads -- Python dictionary containing gradients for each weight matrix and bias vector
    
    learning_rate -- the learning_rate and which we want to go down, a scalar value
    
    
    Returns:
    parameters -- Python dictionary with updated parameters
    """
    
    L = len(parameters) // 2
    for l in range(L):
        parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - (learning_rate * grads["dW"+str(l+1)])
        parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - (learning_rate) * grads["db"+str(l+1)]
    
    return parameters



In [3]:
parameters, grads, learning_rate = update_parameters_with_gd_test_case()

params= update_parameters_with_gd(parameters, grads, learning_rate)
print(params["W1"])
print(params["b1"])
print(params["W2"])
print(params["b2"])

[[ 1.63535156 -0.62320365 -0.53718766]
 [-1.07799357  0.85639907 -2.29470142]]
[[ 1.74604067]
 [-0.75184921]]
[[ 0.32171798 -0.25467393  1.46902454]
 [-2.05617317 -0.31554548 -0.3756023 ]
 [ 1.1404819  -1.09976462 -0.1612551 ]]
[[-0.88020257]
 [ 0.02561572]
 [ 0.57539477]]


## Adam Optimization

Combines the ideas of RMS Prop and Momentum Gradient Descent

1. It calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v^{corrected}$ (with bias correction). 
2. It calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s^{corrected}$ (with bias correction). 
3. It updates parameters in a direction based on combining information from "1" and "2".


$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$

where:
- t counts the number of steps taken of Adam 
- L is the number of layers
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages. 
- $\alpha$ is the learning rate
- $\varepsilon$ is a very small number to avoid dividing by zero

### Initialize Adam

In [6]:
def initialize_adam(parameters):
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v -- python dictionary that will contain the exponentially weighted average of the gradient.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...
    """
    L = len(parameters) // 2
    v = {}
    s = {}
    
    for l in range(L):
        v["dW"+str(l+1)] = np.zeros_like(parameters["W"+str(l+1)])
        v["db"+str(l+1)] = np.zeros_like(parameters["b"+str(l+1)])
        s["dW"+str(l+1)] = np.zeros_like(parameters["W"+str(l+1)])
        s["db"+str(l+1)] = np.zeros_like(parameters["b"+str(l+1)])
    
    return v, s

In [7]:
parameters = initialize_adam_test_case()
v, s = initialize_adam(parameters)
print(v)
print(s)

{'dW1': array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]]), 'db1': array([[ 0.],
       [ 0.]]), 'dW2': array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]]), 'db2': array([[ 0.],
       [ 0.],
       [ 0.]])}
{'dW1': array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]]), 'db1': array([[ 0.],
       [ 0.]]), 'dW2': array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]]), 'db2': array([[ 0.],
       [ 0.],
       [ 0.]])}


### Update Paramteters With Adam

In [9]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Update parameters using Adam
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    t -- Number of steps taken of Adam
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    
    L = len(parameters) // 2
    v_corrected = {}
    s_corrected = {}
    
    for l in range(L):
        ## Moving average of gradients
        v["dW"+str(l+1)] = beta1 * v["dW"+str(l+1)] + (1-beta1)*grads["dW"+str(l+1)]
        v["db"+str(l+1)] = beta1 * v["db"+str(l+1)] + (1-beta1)*grads["db"+str(l+1)]
        
        ## Compute bias corrected first moment estimate
        v_corrected["dW"+str(l+1)] = v["dW"+str(l+1)] / (1-np.power(beta1, t))
        v_corrected["db"+str(l+1)] = v["db"+str(l+1)] / (1-np.power(beta1, t))
        
        ## Moving average of squared gradients
        s["dW"+str(l+1)] = beta2 * s["dW"+str(l+1)] / (1-beta2)*np.power(grads["dW"+str(l+1)], 2)
        s["db"+str(l+1)] = beta2 * s["db"+str(l+1)] / (1-beta2)*np.power(grads["db"+str(l+1)], 2)
        
        ## Bias corrected of second raw moment
        s_corrected["dW"+str(l+1)] = s["dW"+str(l+1)] / (1 - np.power(beta2, t))
        s_corrected["db"+str(l+1)] = s["db"+str(l+1)] / (1 - np.power(beta2, t))
        
        ## Update Parameters
        parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - learning_rate * v_corrected["dW"+str(l+1)]/np.sqrt(s_corrected["dW"+str(l+1)]+epsilon)
        parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - learning_rate * v_corrected["db"+str(l+1)]/np.sqrt(s_corrected["db"+str(l+1)]+epsilon)
        
    
    return parameters, v, s

In [10]:
parameters, grads, v, s = update_parameters_with_adam_test_case()
parameters, v, s  = update_parameters_with_adam(parameters, grads, v, s, t = 2)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))
print("v[\"dW1\"] = " + str(v["dW1"]))
print("v[\"db1\"] = " + str(v["db1"]))
print("v[\"dW2\"] = " + str(v["dW2"]))
print("v[\"db2\"] = " + str(v["db2"]))
print("s[\"dW1\"] = " + str(s["dW1"]))
print("s[\"db1\"] = " + str(s["db1"]))
print("s[\"dW2\"] = " + str(s["dW2"]))
print("s[\"db2\"] = " + str(s["db2"]))

W1 = [[ 59.55167048 -60.86037272 -47.98031494]
 [-27.52003909 -46.54806338  33.6841381 ]]
b1 = [[  8.21271837]
 [ 48.48981595]]
W2 = [[ 14.41841171 -28.16281599  37.8653054 ]
 [ 18.82162386  35.84456701  44.10045309]
 [ 36.46251317  -0.43333343  58.63337962]]
b2 = [[-13.21552672]
 [-87.31579557]
 [-38.47214061]]
v["dW1"] = [[-0.11006192  0.11447237  0.09015907]
 [ 0.05024943  0.09008559 -0.06837279]]
v["db1"] = [[-0.01228902]
 [-0.09357694]]
v["dW2"] = [[-0.02678881  0.05303555 -0.06916608]
 [-0.03967535 -0.06871727 -0.08452056]
 [-0.06712461 -0.00126646 -0.11173103]]
v["db2"] = [[ 0.02344157]
 [ 0.16598022]
 [ 0.07420442]]
s["dW1"] = [[ 0.  0.  0.]
 [ 0.  0.  0.]]
s["db1"] = [[ 0.]
 [ 0.]]
s["dW2"] = [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
s["db2"] = [[ 0.]
 [ 0.]
 [ 0.]]
